In [1]:
#!/usr/bin/env python3
# Copyright 2004-present Facebook. All Rights Reserved.
#  python optim.py -s example1/synth_test.json -e example1
import argparse
import json
import logging
import os
import random
import time
import torch
import numpy as np

import deep_sdf
import deep_sdf.workspace as ws

import pdb

In [2]:
def adjust_learning_rate(initial_lr, optimizer, num_iterations, decreased_by, adjust_lr_every):
    lr = initial_lr * ((1 / decreased_by) ** (num_iterations // adjust_lr_every))
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

def chamfer_distance(p1, p2):
    '''
    Calculate Chamfer Distance between two point sets
    '''
    p1 = p1.unsqueeze(0)
    p2 = p2.unsqueeze(0)

    p1 = p1.repeat(p2.size(1), 1, 1)
    p1 = p1.transpose(0, 1)

    p2 = p2.repeat(p1.size(0), 1, 1)

    # compute distance tensor
    dist = torch.add(p1, torch.neg(p2))
    dist = torch.norm(dist, 2, dim=2)

    dist1, _ = torch.min(dist, dim = 1)
    dist2, _ = torch.min(dist, dim = 0)

    return torch.mean(dist1) + torch.mean(dist2)

In [21]:
class argms:
    def __init__(self):
        self.experiment_directory = "example1"
        self.checkpoint = "latest"
        self.iterations = 1
        self.split_filename = "example1/synth_test.json"
        self.logfile = None
        self.debug = False
        self.quiet = False
        
args = argms()

In [27]:
if __name__ == "__main__":
    
    torch.manual_seed(0)
    # Initialization
    N_MARCHING_CUBE = 64
    lr= 8e-3
    l2reg= False
    regl2 = 1e-3
    decreased_by = 1.5
    adjust_lr_every = 50
    
    
    
    # pick initialization and samples
    # Load collection of all latent codes
    all_codes_path = os.path.join(
        args.experiment_directory,
        ws.latent_codes_subdir,
        'latest.pth')
    all_codes = torch.load(all_codes_path)['latent_codes']['weight']
    ## sphere
    source_id = 999 # zywvjkvz2492e6xpq4hd1jzy2r9lht        # This will be the source shape (ie starting point)
    latent = all_codes[source_id].unsqueeze(0).detach()#.cuda()   #Add .cuda() if you want to run on GPU
    latent.requires_grad = True

    ## torus
    target_id = 2 # 0bucd9ryckhaqtqvbiagilujeqzek4          # This is be the target shape (ie objective)
    latent_target = all_codes[target_id].unsqueeze(0).detach()#.cuda()   #Add .cuda() if you want to run on GPU

     
    
    specs_filename = os.path.join(args.experiment_directory, "specs.json")
    specs = json.load(open(specs_filename))
    arch = __import__("networks." + specs["NetworkArch"], fromlist=["Decoder"])
    latent_size = specs["CodeLength"]
    # Load decoder: this is our black box function
    decoder = arch.Decoder(latent_size, **specs["NetworkSpecs"])
    decoder = torch.nn.DataParallel(decoder)
    saved_model_state = torch.load(
        os.path.join(
            args.experiment_directory, ws.model_params_subdir, args.checkpoint + ".pth"
        ),
        map_location=torch.device('cpu') # Remove this if you want to run on GPU
    )
    decoder.load_state_dict(saved_model_state["model_state_dict"])
    # Optionally: put decoder on GPU
    #decoder = decoder.module.cuda()


    optimizer = torch.optim.Adam([latent], lr=lr)

    losses = []
    lambdas = []


    # Use Adam optimizer, with source as starting point, and a loss defined on meshes
    # latent is the input of our function
    print("Starting optimization:")
    for e in range(int(args.iterations)):

        adjust_learning_rate(lr, optimizer, e, decreased_by, adjust_lr_every)
        
        start = time.time()
        # Get a mesh representation of our current guess: decoder is evaluated at position latent
        # first create mesh running full forward pass
        verts, faces = deep_sdf.mesh.create_mesh_optim(decoder, latent, N=N_MARCHING_CUBE, max_batch=int(2 ** 18))
        end = time.time()
        #print("time to mesh:", end-start)
        # subsample vertices for gradients computations
        verts = verts[torch.randperm(verts.shape[0])]
        verts = verts[0:20000, :]
        start = time.time()
        # forward pass within loss layer
        xyz_upstream = torch.tensor(verts.astype(float), requires_grad = True, dtype=torch.float32)#, device=torch.device('cuda:0')) # For GPU,

        
        # Get a point cloud sampling of the target shape
        verts_target_sample = verts_target[torch.randperm(verts_target.shape[0])]
        verts_target_sample = verts_target_sample[0:20000, :]
        xyz_target = torch.tensor(verts_target_sample.astype(float), requires_grad = False, dtype=torch.float32) # For GPU, add: , device=torch.device('cuda:0'))

        
        # At this point we have 2 outputs for decoder: the target xyz_target, and the current value xyz_upstream
        # The following lines compute a loss and backpropagate
        # compute loss function: Chamfer between current guess (xyz_upstream) and objective (xyz_target)
        loss = chamfer_distance(xyz_upstream, xyz_target)
        print("Loss at iter", e, ":", loss.item(), ", latent norm: ", torch.norm(latent))
        losses.append(loss.detach().cpu().numpy())                                  ## Loss value
        lambdas.append(torch.norm(latent_target-latent).detach().cpu().numpy())     ## Distance in the domain
        
        
        optimizer.zero_grad()
        decoder.eval()
        loss.backward()
        dL_dx_i = xyz_upstream.grad


        optimizer.zero_grad()
        # use vertices to compute full backward pass
        xyz = torch.tensor(verts.astype(float), requires_grad = True, dtype=torch.float32)#, device=torch.device('cuda:0')) # For GPU,
        latent_inputs = latent.expand(xyz.shape[0], -1)
        inputs = torch.cat([latent_inputs, xyz], 1)#.cuda()      #Add .cuda() if you want to run on GPU
        #first compute normals
        pred_sdf = decoder(inputs)
        loss_normals = torch.sum(pred_sdf)
        loss_normals.backward(retain_graph = True)
        normals = xyz.grad/torch.norm(xyz.grad, 2, 1).unsqueeze(-1)
        print("inputs,", inputs.grad, inputs.requires_grad)
        print("xyz", xyz.grad, xyz.requires_grad)
        print("latent", latent_inputs.grad, latent_inputs.requires_grad, latent.grad, latent.requires_grad)


        # now assemble inflow derivative
        optimizer.zero_grad()
        dL_ds_i_fast = -torch.matmul(dL_dx_i.unsqueeze(1), normals.unsqueeze(-1)).squeeze(-1)
        loss_backward = torch.sum(dL_ds_i_fast * pred_sdf)
        if e % 20 == 0 and e > 0:
            regl2 = regl2/2
        if l2reg:
            loss_backward += regl2* torch.mean(latent.pow(2))
        # Backpropagate
        loss_backward.backward()
        print("loss_backward", latent.grad)

        end = time.time()
       # print("time to backward:", end-start)
        # update latent
        # Explicit gradient is accessible via latent.grad
        optimizer.step()

        # 1. objective function value
        print("loss_backward:", loss_backward.item())
        # 2. its derivative function value on current arguments 
        print("\n")
        #print(latent.grad)

    np.save("./data/loss.npy", losses)
    np.save("./data/lambdas.npy", lambdas)
        #print("shape of verts_target, faces_target: ", verts_target.shape, faces_target.shape, xyz_target.shape)
        #raise Exception("Stop");

Starting optimization:


KeyboardInterrupt: 